In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 5, Finished, Available, Finished)

In [4]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 6, Finished, Available, Finished)

In [5]:
df_validated_data=spark.table("validated_transactions")

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 7, Finished, Available, Finished)

In [6]:
spark.sql('''
 select count(*)
 from validated_transactions''').show()

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 8, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
|    2708|
+--------+



In [7]:
df_typed=(

    df_validated_data
    .withColumn("amount",f.col("amount").cast("decimal(10,2)"))
    .withColumn("transaction_ts",f.to_timestamp("transaction_ts"))
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 9, Finished, Available, Finished)

In [17]:
display(df_typed)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b630859a-bfe7-4797-a617-a3b92e2c0c64)

In [8]:
df_uppercase=(
    df_typed
    .withColumn("merchant",f.upper(f.col("merchant")))
    .withColumn("merchant_category",f.upper(f.col("merchant_category")))
    .withColumn("payment_type",f.upper(f.col("payment_type")))
    .withColumn("status",f.upper(f.col("status")))
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 10, Finished, Available, Finished)

In [9]:
window_spec=( Window
           .partitionBy("transaction_id")
           .orderBy(f.col("transaction_ts").desc())
           )

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 11, Finished, Available, Finished)

In [10]:
df_dedup=(
    df_uppercase
    .withColumn("rn",f.row_number().over(window_spec))
    .filter(f.col("rn")==1)
    .drop("rn")
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 12, Finished, Available, Finished)

In [11]:
df_final_silver=(
    df_dedup
    .withColumn("transaction_date",f.to_date("transaction_ts"))
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 13, Finished, Available, Finished)

In [12]:
(
    df_final_silver
    .write.mode("append")
    .format("delta")
    .saveAsTable("silver_transactions")
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 14, Finished, Available, Finished)

In [13]:
spark.sql('''
       select count(*) 
       from silver_transactions''').show()

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 15, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
|    2708|
+--------+



In [14]:
spark.sql('''
select
COUNT(*) AS total_rows,
    COUNT(DISTINCT transaction_id) AS unique_txns
    from silver_transactions''').show()

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 16, Finished, Available, Finished)

+----------+-----------+
|total_rows|unique_txns|
+----------+-----------+
|      2708|       1354|
+----------+-----------+



In [15]:
df_bronze_customer=spark.table("bronze_raw_customers")

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 17, Finished, Available, Finished)

In [19]:
df_typecast=(
    df_bronze_customer
    .withColumn("created_date",f.col("created_date").cast("date"))
    .withColumn("updated_ts",f.to_timestamp("updated_ts"))
    .withColumn("dob",f.col("dob").cast("date"))
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 21, Finished, Available, Finished)

In [20]:
display(df_typecast)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2cd220ca-44d0-42de-8cc1-6354178a4fcb)

In [25]:
df_upper_customer=(
    df_typecast
    .withColumn("account_type",f.upper(f.col("account_type")))
    .withColumn("risk_rating",f.upper(f.col("risk_rating")))
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 27, Finished, Available, Finished)

In [26]:
display(df_upper_customer)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f88f5b2c-d562-4916-9936-16629f225865)

In [31]:
(

    df_upper_customer
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("silver_customers")
)

StatementMeta(, b58341de-6194-4a78-ae3f-df4be88b5a6d, 33, Finished, Available, Finished)